In [319]:
import pandas as pd

In [320]:
kospi1 = pd.read_csv("./코스피200연결.csv", encoding = "euc-kr")
kospi2 = pd.read_csv("./코스피200개별.csv", encoding = "euc-kr")
st13 = pd.read_csv("./코스피주식(2013).csv", encoding = "euc-kr")
st14 = pd.read_csv("./코스피주식(2014).csv", encoding = "euc-kr")
st15 = pd.read_csv("./코스피주식(2015).csv", encoding = "euc-kr")
st16 = pd.read_csv("./코스피주식(2016).csv", encoding = "euc-kr")
st17 = pd.read_csv("./코스피주식(2017).csv", encoding = "euc-kr")
st18 = pd.read_csv("./코스피주식(2018).csv", encoding = "euc-kr")
st19 = pd.read_csv("./코스피주식(2019).csv", encoding = "euc-kr")
st20 = pd.read_csv("./코스피주식(2020).csv", encoding = "euc-kr")
st21 = pd.read_csv("./코스피주식(2021).csv", encoding = "euc-kr")
st22 = pd.read_csv("./코스피주식(2022).csv", encoding = "euc-kr")


In [321]:
kospi1.columns = ['회사명', '종목코드', '회계년도', '거래월', '종가', '자산', '부채', '당기순이익', '현금흐름', '매출액', '비유동자산', '매출총이익', '법인세비용', '이자비용', '감가상각비', '유동자산',
       '유동부채', '현금및현금성자산']
kospi2.columns = ['회사명', '종목코드', '회계년도', '거래월', '종가', '자산', '부채', '당기순이익', '현금흐름', '매출액', '비유동자산', '매출총이익', '법인세비용', '이자비용', '감가상각비', '유동자산',
       '유동부채', '현금및현금성자산']

In [323]:
st13['회계년도'] = '2013/12'
st14['회계년도'] = '2014/12'
st15['회계년도'] = '2015/12'
st16['회계년도'] = '2016/12'
st17['회계년도'] = '2017/12'
st18['회계년도'] = '2018/12'
st19['회계년도'] = '2019/12'
st20['회계년도'] = '2020/12'
st21['회계년도'] = '2021/12'
st22['회계년도'] = '2022/12'


In [325]:
df = pd.concat([st13, st14, st15, st16, st17, st18, st19, st20, st21, st22], axis=0)

In [326]:
df['종목코드'] = df['종목코드'].apply(lambda x: x.lstrip('0'))

In [332]:
kospi = kospi1.fillna(kospi2)

In [334]:
kospi['종목코드'] = kospi['종목코드'].astype(str)

In [335]:
kospi = pd.merge(kospi, df[['종목코드', '회계년도', '상장주식수', '시가총액']], on=['종목코드', '회계년도'], how='left')

In [337]:
kospi['pbr'] = (kospi['종가']*kospi['상장주식수'])/(kospi['자산']-kospi['부채'])
kospi['per'] = kospi['종가']/(kospi['당기순이익']/kospi['상장주식수'])
kospi['pcr'] = kospi['종가']/(kospi['현금흐름']/kospi['상장주식수'])
kospi['psr'] = kospi['종가']/(kospi['매출액']/kospi['상장주식수'])
kospi['roe'] = kospi['당기순이익']/(kospi['자산']-kospi['부채'])
kospi['roa'] = kospi['당기순이익']/kospi['자산']
kospi['roic'] = kospi['당기순이익']/(kospi['자산']-kospi['비유동자산'])
kospi['gpa'] = kospi['매출총이익']/kospi['자산']

In [339]:
# 분자(EBIT)
magic_ebit = kospi['당기순이익'] + kospi['법인세비용'] + kospi['이자비용']

# 분모
magic_cap = kospi['시가총액']
magic_debt = kospi['부채']

## 분모: 여유자금
magic_excess_cash = kospi['유동부채'] - kospi['유동자산'] + kospi[
    '현금및현금성자산']
magic_excess_cash[magic_excess_cash < 0] = 0
magic_excess_cash_final = kospi['현금및현금성자산'] - magic_excess_cash

magic_ev = magic_cap + magic_debt - magic_excess_cash_final

# 이익수익률
magic_ey = magic_ebit / magic_ev
kospi['매직'] = magic_ey

In [340]:
kospi = kospi.dropna()

In [341]:
len(kospi['회사명'].unique())

179